In [1]:
from theano.sandbox import cuda
cuda.use('gpu3')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(3), GPU number 0 is already in use.


In [36]:
%matplotlib inline
import utils; reload(utils)
import re
from utils import *
from keras.preprocessing import sequence
from keras.layers import *
from __future__ import division, print_function

In [3]:
model_path = 'data/imdb/models/'

## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [4]:
from keras.datasets import imdb
idx = imdb.get_word_index()

2072576/2343108 [=========================>....] - ETA: 0s

This is the word list:

In [5]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [6]:
idx2word = {v: k for k, v in idx.iteritems()}

We download the reviews using code copied from keras.datasets:

In [7]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

65339392/65552540 [============================>.] - ETA: 0s

In [8]:
len(x_train)

25000

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [9]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [10]:
idx2word[23022]

'bromwell'

Here's the whole review, mapped from ids to words.

In [15]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

The labels are 1 for positive, 0 for negative.

In [16]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Reduce vocab size by setting rare words to max index.

In [17]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

Look at distribution of lengths of sentences.

In [18]:
lens = np.array(map(len, trn))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

Pad (with zero) or truncate each sentence to make consistent length.

In [21]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [22]:
trn.shape

(25000, 500)

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [ ]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

The [stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 0.883. So we're short of that, but on the right track.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [26]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    Dropout(0.2),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [27]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [28]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 12s - loss: 0.5177 - acc: 0.7087 - val_loss: 0.2892 - val_acc: 0.8895
Epoch 2/4
25000/25000 [==============================] - 12s - loss: 0.2959 - acc: 0.8840 - val_loss: 0.2618 - val_acc: 0.8916
Epoch 3/4
25000/25000 [==============================] - 12s - loss: 0.2622 - acc: 0.8974 - val_loss: 0.2582 - val_acc: 0.8932
Epoch 4/4
25000/25000 [==============================] - 12s - loss: 0.2464 - acc: 0.9050 - val_loss: 0.2712 - val_acc: 0.8878


That's well past the Stanford paper's accuracy - another win for CNNs!

In [29]:
conv1.save_weights(model_path + 'conv1.h5')

In [ ]:
conv1.load_weights(model_path + 'conv1.h5')

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [30]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [33]:
vecs, words, wordidx = load_vectors('data/glove/6B.300d')

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [66]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    #emb/=3
    return emb

In [38]:
emb = create_emb()

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [41]:
model = Sequential([
    Embedding(vocab_size, vecs.shape[1], input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [42]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [43]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 29s - loss: 0.5638 - acc: 0.6918 - val_loss: 0.4073 - val_acc: 0.8378
Epoch 2/2
25000/25000 [==============================] - 29s - loss: 0.4183 - acc: 0.8176 - val_loss: 0.3701 - val_acc: 0.8606


We already have beaten our previous model! But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [44]:
model.layers[0].trainable=True

In [45]:
model.optimizer.lr=1e-4

In [46]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 29s - loss: 0.3808 - acc: 0.8388 - val_loss: 0.3302 - val_acc: 0.8679


As expected, that's given us a nice little boost. :)

In [48]:
model.optimizer.lr=0.001

In [49]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 29s - loss: 0.3301 - acc: 0.8605 - val_loss: 0.3100 - val_acc: 0.8756
Epoch 2/2
25000/25000 [==============================] - 29s - loss: 0.3070 - acc: 0.8715 - val_loss: 0.2969 - val_acc: 0.8773


In [50]:
model.save_weights(model_path+'glove300.h5')

In [51]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 29s - loss: 0.3040 - acc: 0.8709 - val_loss: 0.3198 - val_acc: 0.8611
Epoch 2/2
19008/25000 [=====================>........] - ETA: 4s - loss: 0.2792 - acc: 0.8826

KeyboardInterrupt: 

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

In [164]:
from keras.layers import Merge
from keras.models import Model

We use the functional API to create multiple conv layers of different sizes, and then concatenate them.

In [165]:
graph_in = Input ((vocab_size, 300))
convs = [ ] 
for fsz in range (5, 8): 
    x = Convolution1D(128, fsz, border_mode='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x)
    convs.append(x)

out = Merge(mode="concat")(convs) 
graph = Model(graph_in, out) 

In [166]:
emb = create_emb()

We then replace the conv/max-pool layer in our original CNN with the concatenated conv layers.

In [167]:
model = Sequential ([
    Embedding(vocab_size, 300, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

In [168]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [169]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 103s - loss: 0.6380 - acc: 0.6560 - val_loss: 0.3380 - val_acc: 0.8596
Epoch 2/2
25000/25000 [==============================] - 103s - loss: 0.3671 - acc: 0.8439 - val_loss: 0.2726 - val_acc: 0.8868


In [170]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 103s - loss: 0.3118 - acc: 0.8735 - val_loss: 0.2694 - val_acc: 0.8928
Epoch 2/2
25000/25000 [==============================] - 103s - loss: 0.2874 - acc: 0.8832 - val_loss: 0.2763 - val_acc: 0.8839


Interestingly, I found that in this case I got best results when I started the embedding layer as being trainable, and then set it to non-trainable after a couple of epochs. I have no idea why!

In [171]:
model.layers[0].trainable=False

In [172]:
model.optimizer.lr=1e-5

In [173]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 103s - loss: 0.2757 - acc: 0.8921 - val_loss: 0.2621 - val_acc: 0.8960
Epoch 2/2
25000/25000 [==============================] - 103s - loss: 0.2509 - acc: 0.8996 - val_loss: 0.2585 - val_acc: 0.9000


This more complex architecture has given us another boost in accuracy.

In [174]:
model.save_weights(model_path+'glove300-multi-cnn-128-5-8.h5')

In [175]:
for layer in model.layers:
    layer.trainable = False

In [176]:
model.layers[-1].trainable = True
model.layers[-3].trainable = True

In [177]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 103s - loss: 0.2428 - acc: 0.9039 - val_loss: 0.2522 - val_acc: 0.8992
Epoch 2/2
25000/25000 [==============================] - 103s - loss: 0.2229 - acc: 0.9140 - val_loss: 0.2469 - val_acc: 0.9000


In [127]:
model.save_weights(model_path+'glove300-multi-cnn.h5.p2')

## LSTM

We haven't covered this bit yet!

In [178]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              W_regularizer=l2(1e-6), dropout=0.2),
    BatchNormalization(),
    LSTM(100, consume_less='gpu'),
    Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_15 (Embedding)         (None, 500, 32)       160000      embedding_input_15[0][0]         
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 500, 32)       64          embedding_15[0][0]               
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 100)           53200       batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_29 (Dense)                 (None, 1)             101         lstm_1[0][0]                     
Total params: 213365
______________________________________________________________________

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

In [ ]:
model.optimizer.lr = 0.001